# Let's get some predictions!

In [3]:
#packages
import polars as pl
import pickle
import numpy as np
import pandas as pd
import xlsx2csv
from sklearn.ensemble import RandomForestClassifier

pl.Config.set_fmt_str_lengths(150);
pl.Config.set_tbl_cols(40);

In [4]:
#set up R extension
%load_ext rpy2.ipython

c:\Users\e_sut\anaconda3\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [90]:
#set current values
season = 2024
roundNum = 11

### Update based on last round (unless last round is round 0) and bring in other useful things

In [91]:
#Get Team index

#load team array
indexTeams = pl.read_excel('indexTeams.xlsx').with_columns(pl.col('team.A.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))

In [73]:
#Get distnace travelled dataframe

#load in distance array
distanceTravelled = pl.read_excel('distanceTravelled.xlsx').with_columns(pl.col('team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))

In [92]:
#get results of past round
if roundNum != 0:
    pastRound = roundNum - 1
else :
    pastRound = roundNum

In [93]:
%%R -i season,pastRound -o results

library(fitzRoy)
library('dplyr')
library('tidyr')

results <- fetch_results_afl(season=season, round_number=pastRound) %>% select(match.date, round.roundNumber, venue.name, match.homeTeam.name, match.awayTeam.name,
                                                                               round.year, homeTeamScore.matchScore.totalScore, homeTeamScore.matchScore.behinds,
                                                                               homeTeamScore.matchScore.goals, awayTeamScore.matchScore.totalScore, awayTeamScore.matchScore.behinds,
                                                                               awayTeamScore.matchScore.goals)      

In [94]:
results = (pl.from_pandas(results)
            .with_columns(pl.col('match.date').dt.date())
            .rename({'match.date' : 'Date',
                      'round.year' : 'compSeason.year',
                      'homeTeamScore.matchScore.totalScore' : 'home.team.totalScore',
                      'homeTeamScore.matchScore.behinds' : 'home.team.behinds',
                      'homeTeamScore.matchScore.goals' : 'home.team.goals',
                      'awayTeamScore.matchScore.totalScore' : 'away.team.totalScore',
                      'awayTeamScore.matchScore.behinds' : 'away.team.behinds',
                      'awayTeamScore.matchScore.goals' : 'away.team.goals',
                      'match.homeTeam.name' : 'home.team.name',
                      'match.awayTeam.name' : 'away.team.name'})
             #cast to int64
             .with_columns(pl.col('compSeason.year').cast(pl.Int64),
                           pl.col('round.roundNumber').cast(pl.Int64),
                           pl.col('home.team.totalScore').cast(pl.Int64),
                           pl.col('home.team.behinds').cast(pl.Int64),
                           pl.col('home.team.goals').cast(pl.Int64),
                           pl.col('away.team.totalScore').cast(pl.Int64),
                           pl.col('away.team.behinds').cast(pl.Int64),
                           pl.col('away.team.goals').cast(pl.Int64))
             #convert any team names that require as much
             .with_columns(pl.col('home.team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
             .with_columns(pl.col('away.team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
             #Add the indexs
             .join(indexTeams, how='left', left_on='home.team.name', right_on='team.A.name')
             .rename({'EloIndex' : 'home.team.id'})
             .join(indexTeams, how='left', left_on='away.team.name', right_on='team.A.name')
             .rename({'EloIndex' : 'away.team.id'})
        )

In [441]:
#create a function to find a teams prediction
def eloTeamPrediction(ratingTeamA, ratingTeamB):
    exp = (-1*(ratingTeamA - ratingTeamB))/400
    b = 1 + 10 ** exp
    result = 1/b
    return result

In [442]:
#create a function to find the teams result, inital B = 0.004 per Multifactorial analysis of factors influencing elite australian football match outcomes: a machine learning approach
def eloTeamResult(scoreTeamA, scoreTeamB):
    exp = -0.04*(scoreTeamA - scoreTeamB)
    b = 1 + np.exp(exp)
    result = 1/b
    return result

In [443]:
#Create a function to get change in teams elo, set K=67.559 per Multifactorial analysis of factors influencing elite australian football match outcomes: a machine learning approach
def eloChange(scoreTeamA, scoreTeamB, ratingTeamA, ratingTeamB):
    #get Team prediction
    prediction = eloTeamPrediction(ratingTeamA, ratingTeamB)
    
    #get actual result
    actual = eloTeamResult(scoreTeamA , scoreTeamB)
    
    #get elo change
    result = 20*(actual - prediction)
    
    #get new elo
    newElo = ratingTeamA + result

    return newElo

In [444]:
def eloNumber(indexA, indexB, scoreA, scoreB):

        #Get the new elo score for both teams
        eloA = eloChange(scoreA, scoreB, eloScores[indexA], eloScores[indexB])
        eloB = eloChange(scoreB, scoreA, eloScores[indexB], eloScores[indexA])
        #Update list of list to reflect the new score
        eloScores[indexA] = eloA
        eloScores[indexB] = eloB
        

In [445]:
#Update elo scores
#load the current elo score
with open("eloScores", "rb") as fp:   # Unpickling
    eloScores = pickle.load(fp)


#update based on past round
if roundNum != 0:
    results.with_columns(pl.struct(['home.team.id', 'away.team.id', 'home.team.totalScore', 'away.team.totalScore'])
                           .map_elements(lambda x: eloNumber(x['home.team.id'], x['away.team.id'], x['home.team.totalScore'], x['away.team.totalScore'])))
#save updated elo scores
with open("eloScores", "wb") as fp:
    pickle.dump(eloScores, fp)

In [446]:
#Get match date & update

#load in match dates
matchDays = pl.read_excel('matchDay.xlsx')
#update based on teams that played
if roundNum != 0:
    matchDays = (matchDays
                    .with_columns(pl.col('Team').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                    .join(results.melt(id_vars='Date', value_vars=['home.team.name', 'away.team.name'], value_name='team.name').select('Date', 'team.name'),
                          how='left',
                          left_on='Team',
                          right_on='team.name')
                     .with_columns(pl.when(pl.col('Date_right').is_null()).then(pl.col('Date')).otherwise(pl.col('Date_right')).alias('Date'))
                     .drop('Date_right')
                )
    
#save updated version
matchDays.write_excel('matchDay.xlsx')

In [95]:
#Get all matches both sides & update

#load in past years matches both sides
matchesBothSides = pl.read_excel('matchesBothSides.xlsx').drop('round.Id_right')
print(matchesBothSides.columns)
#update based on past results
if roundNum != 0:
    matchesBothSides = (matchesBothSides
                        .with_columns(pl.col('team.A.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']),
                                        pl.col('team.B.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                         .drop('round.Id')
                         .vstack((results
                         .select('compSeason.year', 'round.roundNumber', 'venue.name', 'home.team.id', 'home.team.name', 'home.team.totalScore',
                                 'home.team.goals', 'home.team.behinds', 'away.team.id', 'away.team.name', 'away.team.totalScore', 'away.team.goals', 'away.team.behinds')
                         .rename({'home.team.name': 'team.A.name',
                                  'home.team.id' : 'team.A.club.id',
                                  'home.team.totalScore' : 'team.A.score.totalScore',
                                  'home.team.goals' : 'team.A.score.goals',
                                  'home.team.behinds' : 'team.A.score.behinds',
                                  'away.team.name': 'team.B.name',
                                  'away.team.id' : 'team.B.club.id',
                                  'away.team.totalScore' : 'team.B.score.totalScore',
                                  'away.team.goals' : 'team.B.score.goals',
                                  'away.team.behinds' : 'team.B.score.behinds'})
                         .vstack(results
                                    .select('compSeason.year', 'round.roundNumber', 'venue.name', 'away.team.id', 'away.team.name', 'away.team.totalScore',
                                            'away.team.goals', 'away.team.behinds', 'home.team.id', 'home.team.name', 'home.team.totalScore', 'home.team.goals', 'home.team.behinds')
                                    .rename({'home.team.name': 'team.B.name',
                                              'home.team.id' : 'team.B.club.id',
                                              'home.team.totalScore' : 'team.B.score.totalScore',
                                              'home.team.goals' : 'team.B.score.goals',
                                              'home.team.behinds' : 'team.B.score.behinds',
                                              'away.team.name': 'team.A.name',
                                              'away.team.id' : 'team.A.club.id',
                                              'away.team.totalScore' : 'team.A.score.totalScore',
                                              'away.team.goals' : 'team.A.score.goals',
                                              'away.team.behinds' : 'team.A.score.behinds'}))
                                        ))
                         .sort('compSeason.year', 'round.roundNumber')   
                        )
#sort out the round id
roundId = (matchesBothSides
             .select('compSeason.year', 'round.roundNumber')
             .unique()
             .sort('compSeason.year', 'round.roundNumber')
             .with_columns(pl.col('round.roundNumber').cum_count().alias('round.Id'))
          )
#Add in the roundId
matchesBothSides = (matchesBothSides
                      #.drop('round.Id') #remove for round 0
                      .join(roundId, how='left', on=['compSeason.year', 'round.roundNumber']))
#save updated version
matchesBothSides.write_excel('matchesBothSides.xlsx')

['compSeason.year', 'round.roundNumber', 'venue.name', 'team.A.club.id', 'team.A.name', 'team.A.score.totalScore', 'team.A.score.goals', 'team.A.score.behinds', 'team.B.club.id', 'team.B.name', 'team.B.score.totalScore', 'team.B.score.goals', 'team.B.score.behinds', 'round.Id']


compSeason.year,round.roundNumber,venue.name,team.A.club.id,team.A.name,team.A.score.totalScore,team.A.score.goals,team.A.score.behinds,team.B.club.id,team.B.name,team.B.score.totalScore,team.B.score.goals,team.B.score.behinds,round.Id
i64,i64,str,i64,str,i64,i64,i64,i64,str,i64,i64,i64,u32
2024,10,"""TIO Stadium""",9,"""Geelong Cats""",100,15,10,4,"""Gold Coast Suns""",164,26,8,39
2024,10,"""SCG""",0,"""Carlton""",65,9,11,16,"""Sydney Swans""",117,17,15,39
2024,10,"""MCG""",1,"""Adelaide Crows""",74,11,8,6,"""Collingwood""",78,12,6,39
2024,10,"""ENGIE Stadium""",17,"""Western Bulldogs""",70,8,22,2,"""GWS Giants""",43,6,7,39
2024,10,"""Marvel Stadium""",3,"""Fremantle""",72,9,18,13,"""St Kilda""",55,8,7,39
2024,10,"""Gabba""",14,"""Richmond""",44,6,8,7,"""Brisbane Lions""",163,26,7,39
2024,10,"""Marvel Stadium""",12,"""North Melbourne""",66,10,6,5,"""Essendon""",106,16,10,39
2024,10,"""Adelaide Oval""",10,"""Hawthorn""",79,12,7,11,"""Port Adelaide""",80,11,14,39
2024,10,"""Optus Stadium""",8,"""Melbourne""",70,10,10,15,"""West Coast Eagles""",105,16,9,39


In [448]:
matchesBothSides = matchesBothSides.with_columns(pl.col('team.A.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']),
                                   pl.col('team.B.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))

### 1. Basic Info first

In [449]:
%%R -i season,roundNum -o fixture,fixtureBye

library('tidyverse')

fixture <- fetch_fixture(season=season, round_number=roundNum)

if (roundNum != 0){
    fixtureBye <- fetch_fixture(season=season, round_number=(roundNum-1)) %>% 
                        select(compSeason.id, compSeason.year, round.roundNumber, round.byes) %>%
                        unnest(round.byes) %>%
                        distinct() %>%
                        mutate(round.currentRound = round.roundNumber + 1,
                               ByeFlag = 1)

    if (nrow(fixtureBye) == 0){

        fixtureBye <- 0 
    }else{
        fixturebye <- fixtureBye %>%
                        select(round.currentRound, club.name, ByeFlag)
    }
}else{
fixtureBye <- 0
}

fixture <- fixture %>% select(-round.byes)

i Returning data for "Round 11, 2024"


v Returning data for "Round 11, 2024" ... done

i Returning data for "Round 10, 2024"
v Returning data for "Round 10, 2024" ... done



In [450]:
fixtureBye

array([0.])

In [451]:
#Add in bye flag if exists (i.e. not first round of the season)
if roundNum != 0 :
    #check if any byes previous round
    if isinstance(fixtureBye, pd.DataFrame):
        #Clean and prep the season bye info
        fixtureBye = (pl.from_pandas(fixtureBye)
                          .drop('club.id')
                          .with_columns(pl.col('round.currentRound').cast(pl.Int32))
                          #convert any team names that require as much
                          .with_columns(pl.col('name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']),
                                   pl.col('club.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                     )

In [477]:
#The inital cleaning working with the gneral match details. We can use this as our base for creating the desired attributes
if roundNum != 0 and isinstance(fixtureBye, pd.DataFrame):
    fixtureBase = (pl.from_pandas(fixture)
                .sort('round.id')
                #Keep only the columns we want
                .select('round.roundNumber', 'utcStartTime', 'home.team.name',
                        'away.team.name', 'venue.id', 'venue.name', 'venue.state')
                #convert any team names that require as much
                .with_columns(pl.col('home.team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                .with_columns(pl.col('away.team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                #Create the date in format wanted
                .with_columns(pl.col('utcStartTime').str.slice(0, length=10).str.to_date().alias('Date'))
                .drop('utcStartTime')
                #Add a bye flag
                .join(fixtureBye.select('round.currentRound', 'club.name', 'ByeFlag'), how='left', left_on=['round.roundNumber', 'home.team.name'], right_on=[ 'round.currentRound', 'club.name'])
                .join(fixtureBye.select('round.currentRound', 'club.name', 'ByeFlag'), how='left', left_on=['round.roundNumber', 'away.team.name'], right_on=['round.currentRound', 'club.name'])
                #Rename bye flag
                .rename({'ByeFlag' : 'home.bye.flag',
                         'ByeFlag_right' : 'away.bye.flag'})
                #Fill in bye flag nulls
                .with_columns(pl.col('home.bye.flag').fill_null(0).cast(pl.Int32),
                              pl.col('away.bye.flag').fill_null(0).cast(pl.Int32))
                #Fill venue blank states with china (1 is not china but NZ but is in 2015 so we wont worry about that)
                .with_columns(pl.col('venue.state').fill_null('China'))
                #Add team index
                .join(indexTeams.drop('team.A.club.id'), how='left', left_on='home.team.name', right_on='team.A.name')
                .rename({'EloIndex' : 'home.team.id'})
                .join(indexTeams.drop('team.A.club.id'), how='left', left_on='away.team.name', right_on='team.A.name')
                .rename({'EloIndex' : 'away.team.id'})
                )
else:
    #First round of the season
        fixtureBase = (pl.from_pandas(fixture)
                .sort('round.id')
                  #Keep only the columns we want
                .select('round.roundNumber', 'utcStartTime', 'home.team.name',
                        'away.team.name', 'venue.id', 'venue.name', 'venue.state')
                #convert any team names that require as much
                .with_columns(pl.col('home.team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                .with_columns(pl.col('away.team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                 #Create the date in format wanted
                 .with_columns(pl.col('utcStartTime').str.slice(0, length=10).str.to_date().alias('Date'))
                 .drop('utcStartTime')
                 #Add a bye flag
                 .with_columns(pl.lit(0).alias('home.bye.flag'),
                               pl.lit(1).alias('away.bye.flag'))
                #Fill venue blank states with china (1 is not china but NZ but is in 2015 so we wont worry about that)
                .with_columns(pl.col('venue.state').fill_null('China'))
                #Add team index
                .join(indexTeams.drop('team.A.club.id'), how='left', left_on='home.team.name', right_on='team.A.name')
                .rename({'EloIndex' : 'home.team.id'})
                .join(indexTeams.drop('team.A.club.id'), how='left', left_on='away.team.name', right_on='team.A.name')
                .rename({'EloIndex' : 'away.team.id'})
                )

### 2. Add in current ladder position

In [478]:
%%R -i season,roundNum -o ladder

if (roundNum != 0){
    ladder <- fetch_ladder_afl(season=season, round_number=roundNum-1)
}else{
    ladder <- fetch_ladder_afl(season=season-1)
}

ladder <- ladder %>% select('team.name', 'position')

In [479]:
#from pandas to polars
ladder = (pl.from_pandas(ladder)
            #convert any team names that require as much
             .with_columns(pl.col('team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
         )

In [480]:
#join to basic fixture info
fixtureLadder = (fixtureBase
                        #Home team ladder
                        .join(ladder, how='left', left_on='home.team.name', right_on='team.name')
                        .rename({'position' : 'home.ladder.position'})
                        #Away team ladder
                        .join(ladder, how='left', left_on='away.team.name', right_on='team.name')
                        .rename({'position' : 'away.ladder.position'})                    
                    )

### 3. Add the distance travelled

In [481]:
#use loaded Distnace Travelled dataframe
fixtureDT = (fixtureLadder 
                 #Home team distance travelled to venue
                 .join(distanceTravelled.rename({'Distance' : 'home.distance'}), how='left', left_on=['home.team.name', 'venue.state'], right_on=['team.name', 'State'])
                 #Away team distance travelled to venue
                 .join(distanceTravelled.rename({'Distance' : 'away.distance'}), how='left', left_on=['away.team.name', 'venue.state'], right_on=['team.name', 'State'])
                )

### 4. Add Elo Rating

In [482]:
#use updated elo score to populate
fixtureElo = (fixtureDT
                 .with_columns(pl.col('home.team.id').map_elements(lambda x : eloScores[x]).alias('home.elo.score'),
                               pl.col('away.team.id').map_elements(lambda x : eloScores[x]).alias('away.elo.score'))
                )

### 5. Add in Home Adavantage/Disadvantage

In [483]:
#Use matches both sides to figure out last 20 games if played at venue
venuePortions = (matchesBothSides
                     .select('compSeason.year', 'round.roundNumber', 'venue.name', 'team.A.name')
                     .with_columns(pl.col('venue.name').count().over('team.A.name', 'venue.name').alias('at.venue'),
                                   pl.col('team.A.name').count().over('team.A.name').alias('game.count'))
                     .with_columns((pl.col('at.venue')/pl.col('game.count')).alias('venue.portion'))
                     .select('team.A.name', 'venue.name', 'venue.portion')
                     .unique()     
                    )
fixtureHome = (fixtureElo
                   #home team
                   .join(venuePortions, how='left', left_on=['home.team.name', 'venue.name'], right_on=['team.A.name', 'venue.name'])
                   .rename({'venue.portion' : 'home.venue.portion'})
                   .with_columns(pl.col('home.venue.portion').fill_null(0))
                   #away team
                   .join(venuePortions, how='left', left_on=['away.team.name', 'venue.name'], right_on=['team.A.name', 'venue.name'])
                   .rename({'venue.portion' : 'away.venue.portion'})
                   .with_columns(pl.col('away.venue.portion').fill_null(0))
               
              )

### 6. Add Days since last match

In [485]:
#Use days since last match to populate
fixtureDays = (fixtureHome
                   #home teams
                   .join(matchDays, how='left', left_on='home.team.name', right_on='Team')
                   .rename({'Date_right' : 'home.last.date'})
                   .with_columns((pl.col('Date') - pl.col('home.last.date').str.to_date()).dt.total_days().alias('home.day.last.match'))
                   .drop('home.last.date')
                    #away teams
                   .join(matchDays, how='left', left_on='away.team.name', right_on='Team')
                   .rename({'Date_right' : 'away.last.date'})
                   .with_columns((pl.col('Date') - pl.col('away.last.date').str.to_date()).dt.total_days().alias('away.day.last.match'))
                   .drop('away.last.date')
 
                    )

### 7. Add Attacking/Defending/Opposition Numbers

In [491]:
#Get the goals scored and given away and the margin of each game. Add in the ladder positions as well
matchesBothSidesStats = (matchesBothSides
                 .with_columns(pl.col('team.A.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                        new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']),
                                pl.col('team.B.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                        new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                 .unique()
                 #Goals scored
                 .with_columns((pl.col('team.A.score.goals') + (pl.col('team.A.score.behinds')*0.5)).alias('strength.attack'))
                 #Goals given away
                 .with_columns((pl.col('team.B.score.goals') + (pl.col('team.B.score.behinds')*0.5)).alias('strength.defense'))
                 #goal diff , i.e. margin
                 #.with_columns((pl.col('team.A.score.totalScore') - pl.col('team.B.score.totalScore')).abs().alias('margin'))
                 #keep only what I want
                 .select('compSeason.year', 'round.roundNumber', 'round.Id', 'team.A.club.id', 'team.A.name', 'team.B.club.id', 'team.B.name',
                         'strength.attack', 'strength.defense', 'team.A.score.totalScore', 'team.B.score.totalScore')
                 #Cast round.id to int rather an unsigned int
                 .with_columns(pl.col('round.Id').cast(pl.Int32))
            )   

In [496]:
#Get the goals scored and given away and the margin of each game. Add in the ladder positions as well
matchStats = (matchesBothSides
                .with_columns(pl.col('team.A.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                        new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']),
                                pl.col('team.B.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                        new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
                .unique()
                 #Goals scored
                 .with_columns((pl.col('team.A.score.goals') + (pl.col('team.A.score.behinds')*0.5)).alias('strength.attack'))
                 #Goals given away
                 .with_columns((pl.col('team.B.score.goals') + (pl.col('team.B.score.behinds')*0.5)).alias('strength.defense'))
                 #keep only what I want
                 .select('compSeason.year', 'round.roundNumber', 'round.Id', 'team.A.club.id', 'team.A.name', 'team.B.club.id', 'team.B.name',
                         'strength.attack', 'strength.defense', 'team.A.score.totalScore', 'team.B.score.totalScore')
                 #Cast round.id to int rather an unsigned int
                 .with_columns(pl.col('round.Id').cast(pl.Int32))
                 #Get the each teams rolling 5 game strength based on margin
                 #get the previous 5 games margin average of opposition
                 #Start with descending round.id for rolling groupby statements
                 .sort(['team.A.name', 'round.Id'], descending=[False, True])
                 #Try a rolling previous avg margin
                 .sort('team.A.name', 'round.Id')
                 .rolling(index_column='round.Id', period='5i', by='team.A.name')
                 .agg(pl.col('team.A.score.totalScore').sum().alias('rolling.team.A.totalScore'),
                 pl.col('team.B.score.totalScore').sum().alias('rolling.team.B.totalScore'))
                 #get the avg margin
                 .with_columns(((pl.col('rolling.team.A.totalScore') - pl.col('rolling.team.B.totalScore'))/5).alias('rolling.margin.avg'))
                 #join back up to df
                 .join(matchesBothSidesStats, how='left', on=['team.A.name', 'round.Id'])
                 #arrange columns as desired
                 .select('compSeason.year', 'round.Id', 'team.A.name', 'strength.attack',
                         'strength.defense', 'rolling.margin.avg')
                 #Get the previous 5 matches attacking strength
                 .with_columns(pl.col('strength.attack').over('team.A.name').alias('strength.attack.t-1'),
                               pl.col('strength.attack').shift(1).over('team.A.name').alias('strength.attack.t-2'),
                               pl.col('strength.attack').shift(2).over('team.A.name').alias('strength.attack.t-3'),
                               pl.col('strength.attack').shift(3).over('team.A.name').alias('strength.attack.t-4'),
                               pl.col('strength.attack').shift(4).over('team.A.name').alias('strength.attack.t-5'))
                 #Get the previous 5 matches defending strength
                 .with_columns(pl.col('strength.defense').over('team.A.name').alias('strength.defense.t-1'),
                               pl.col('strength.defense').shift(1).over('team.A.name').alias('strength.defense.t-2'),
                               pl.col('strength.defense').shift(2).over('team.A.name').alias('strength.defense.t-3'),
                               pl.col('strength.defense').shift(3).over('team.A.name').alias('strength.defense.t-4'),
                               pl.col('strength.defense').shift(4).over('team.A.name').alias('strength.defense.t-5'))
                 #drop the original strength metrics
                 .drop('strength.attack', 'strength.defense')
                            
                 #Will need to grab only the most resent results
                 .sort('compSeason.year', 'round.Id')
                 .with_columns(pl.col('round.Id').max().over(pl.col('team.A.name')).alias('max.round'))
                 .filter(pl.col('max.round') == pl.col('round.Id'))
                 #remove unwanted columns
                 .drop('compSeason.year', 'round.Id', 'max.round')

                    
                )

In [497]:
fixtureADO = (fixtureDays
     #join for home team
     .join(matchStats, how='left',
           left_on= 'home.team.name',
           right_on= 'team.A.name')
 
     #get past strengths of home team
     .rename({'strength.attack.t-1' : 'home.attack.t-1',
              'strength.attack.t-2' : 'home.attack.t-2',
              'strength.attack.t-3' : 'home.attack.t-3',
              'strength.attack.t-4' : 'home.attack.t-4',
              'strength.attack.t-5' : 'home.attack.t-5',
              'strength.defense.t-1' : 'home.defense.t-1',
              'strength.defense.t-2' : 'home.defense.t-2',
              'strength.defense.t-3' : 'home.defense.t-3',
              'strength.defense.t-4' : 'home.defense.t-4',
              'strength.defense.t-5' : 'home.defense.t-5',
              #For the away team the oppositions strength
              'rolling.margin.avg' : 'away.opposition.strength'})
 
      #join for away team
      .join(matchStats, how='left',
           left_on='away.team.name',
           right_on='team.A.name')
 
      #get past strengths of away team
      .rename({'strength.attack.t-1' : 'away.attack.t-1',
              'strength.attack.t-2' : 'away.attack.t-2',
              'strength.attack.t-3' : 'away.attack.t-3',
              'strength.attack.t-4' : 'away.attack.t-4',
              'strength.attack.t-5' : 'away.attack.t-5',
              'strength.defense.t-1' : 'away.defense.t-1',
              'strength.defense.t-2' : 'away.defense.t-2',
              'strength.defense.t-3' : 'away.defense.t-3',
              'strength.defense.t-4' : 'away.defense.t-4',
              'strength.defense.t-5' : 'away.defense.t-5',
              #For the home team the oppositions strength
              'rolling.margin.avg' : 'home.opposition.strength'})
            
    )

### 8. Add Player Rankings

In [463]:
%%R -o players
seasonList <- c(2023, 2024)
first <- TRUE
for(s in seasonList) {
    
    df <- fetch_player_stats(season=s) %>% mutate('compSeason.year' = s) %>% select(compSeason.shortName, compSeason.year, round.roundNumber, teamId, team.name, player.playerId, player.givenName, player.surname, ratingPoints)

    if (first) {
        players <- df
        first <- FALSE
    }else{
        players <- players %>% 
                      bind_rows(df)
    }
}
    

i Fetching match ids


v Fetching match ids ... done

i Finding player stats for 216 matches.
v Finding player stats for 216 matches. ... done

i Fetching match ids
v Fetching match ids ... done

i Finding player stats for 207 matches.
v Finding player stats for 207 matches. ... done



In [464]:
players = (pl.from_pandas(players)
             #convert any team names that require as much
             .with_columns(pl.col('team.name').replace(old=['GWS GIANTS', 'Gold Coast SUNS', 'Narrm', 'Yartapuulti', 'Walyalup', 'Euro-Yroke', 'Waalitj Marawar', 'Kuwarna'],
                                          new=['GWS Giants', 'Gold Coast Suns', 'Melbourne', 'Port Adelaide', 'Fremantle', 'St Kilda', 'West Coast Eagles', 'Adelaide Crows']))
          )

In [499]:
playerRoundIndex = (players
     .filter(~pl.col('team.name').is_null())
     .select('compSeason.year', 'round.roundNumber')
     .unique()
     .sort(['compSeason.year', 'round.roundNumber'])
     .with_columns(pl.col('round.roundNumber').cum_count().cast(pl.Int32).alias('round.Id'))
)

In [466]:
#Create the base of the player ratings    FOR FUTURE IMPLINTATION
#playerRatings = (players
#                     .filter(~pl.col('player.playerId').is_null())
#                     .join(playerRoundIndex, how='left', on=['compSeason.year', 'round.roundNumber'])
#                     .sort('player.playerId', 'round.Id')
#get sum of past 10 games
#                     .rolling(index_column='round.Id', period='10i', by=['player.playerId', 'player.givenName', 'player.surname'])
#                     .agg(pl.col('ratingPoints').sum().alias('rolling.player.points'))
#                     .sort('player.playerId', 'round.Id')
#                     #get the next round to join on
#                     .with_columns(pl.col('round.Id').shift(-1).over('player.playerId').alias('round.nextRound'))
#                    #join back up to get the team
#                     .join(playerRoundIndex, how='left', on='round.Id')
#                     .join(players.select('compSeason.year', 'round.roundNumber', 'teamId', 'team.name', 'player.playerId'),
#                           on=['compSeason.year', 'round.roundNumber', 'player.playerId'])
#                     .sort('player.playerId', 'round.Id')
#                )

In [500]:
#get the sum of teams rating
sumedteamRating = (players
    .join(playerRoundIndex, how='left', on=['compSeason.year', 'round.roundNumber'])
    .filter(~pl.col('team.name').is_null())
    .group_by('team.name', 'round.Id')
    .agg(pl.col('ratingPoints').sum())                   
    .sort('team.name', 'round.Id')
    .rolling(index_column='round.Id', period='10i', by=['team.name'])
    .agg(pl.col('ratingPoints').sum().alias('rolling.ratingPoints'))
    .with_columns(pl.col('round.Id').max().over('team.name').alias('lastRound'))
    #only want most recent results
    .filter(pl.col('round.Id') == pl.col('lastRound'))
    #keep only what we want
    .select('team.name', 'rolling.ratingPoints')
)

In [501]:
fixturePlayer = (fixtureADO     
     #join home teams ratings
     .join(sumedteamRating, how='left', left_on=['home.team.name'], right_on=['team.name'])
     .rename({'rolling.ratingPoints' : 'home.team.playerPoints'})
     #join away tea,s ratings
     .join(sumedteamRating, how='left', left_on=['away.team.name'], right_on=['team.name'])
     .rename({'rolling.ratingPoints' : 'away.team.playerPoints'})
)

### 9. Put everything in order

In [502]:
fixtureReady = (fixturePlayer
             .select('home.bye.flag', 'home.ladder.position', 'home.elo.score', 'home.venue.portion', 'home.day.last.match', 'home.attack.t-1', 'home.attack.t-2',
                   'home.attack.t-3', 'home.attack.t-4', 'home.attack.t-5', 'home.defense.t-1', 'home.defense.t-2', 'home.defense.t-3', 'home.defense.t-4', 'home.defense.t-5', 'home.opposition.strength',
                   'home.team.playerPoints', 'away.bye.flag', 'away.ladder.position', 'away.elo.score', 'away.venue.portion', 'away.day.last.match', 'away.attack.t-1', 'away.attack.t-2',
                   'away.attack.t-3', 'away.attack.t-4', 'away.attack.t-5', 'away.defense.t-1', 'away.defense.t-2', 'away.defense.t-3', 'away.defense.t-4', 'away.defense.t-5', 'away.opposition.strength',
                   'away.team.playerPoints')            
            )
predictionReady = (fixturePlayer
                      .select('home.team.name', 'away.team.name'))

In [503]:
fixtureReady

home.bye.flag,home.ladder.position,home.elo.score,home.venue.portion,home.day.last.match,home.attack.t-1,home.attack.t-2,home.attack.t-3,home.attack.t-4,home.attack.t-5,home.defense.t-1,home.defense.t-2,home.defense.t-3,home.defense.t-4,home.defense.t-5,home.opposition.strength,home.team.playerPoints,away.bye.flag,away.ladder.position,away.elo.score,away.venue.portion,away.day.last.match,away.attack.t-1,away.attack.t-2,away.attack.t-3,away.attack.t-4,away.attack.t-5,away.defense.t-1,away.defense.t-2,away.defense.t-3,away.defense.t-4,away.defense.t-5,away.opposition.strength,away.team.playerPoints
i32,i32,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,11,1510.957192,0.521739,271,20.0,18.5,14.0,25.0,16.5,17.5,18.0,15.5,15.5,17.0,3.2,2198.8,1,1,1573.671311,0.057143,77,19.0,16.0,14.0,15.0,21.0,14.0,15.0,16.5,16.5,19.0,13.0,2050.6
0,9,1476.869427,0.521739,272,18.5,13.0,27.0,15.0,15.5,12.0,15.0,8.5,16.5,14.0,-4.0,1969.1,1,5,1459.930332,0.027027,76,19.0,21.0,13.0,12.0,18.5,21.0,17.0,12.5,12.5,9.5,25.2,1909.3
0,18,1443.099033,0.173913,273,26.0,16.0,14.5,15.5,13.5,20.5,22.5,17.0,21.5,16.0,5.4,1896.6,1,4,1527.292994,0.04,252,17.0,15.5,21.0,15.0,26.0,20.5,23.5,15.5,13.0,16.5,-8.0,2030.3
0,10,1504.690582,0.216216,78,17.0,13.5,14.5,15.0,14.5,18.5,17.5,17.5,16.0,20.5,7.8,1965.7,1,8,1556.018935,0.088235,77,21.5,20.5,17.5,19.0,12.5,12.0,26.0,19.5,21.0,18.5,-1.4,2001.8
0,3,1519.399466,0.391304,273,17.5,11.5,20.5,20.5,14.0,20.0,20.0,22.5,18.5,15.5,15.6,2199.8,1,6,1578.752013,0.027027,77,21.0,12.5,20.5,20.5,20.5,19.0,13.0,17.0,16.5,14.5,-12.2,1995.3
0,17,1462.941596,0.382353,77,12.0,15.5,22.5,12.5,15.5,21.5,21.0,16.0,18.5,25.0,-37.6,1625.5,1,2,1489.203434,0.347826,274,9.5,8.0,17.0,16.5,19.5,18.5,31.0,14.5,14.0,20.5,-7.8,2081.1
0,15,1479.65997,0.173913,274,12.0,12.0,15.5,20.5,18.5,18.5,17.5,14.0,14.5,23.0,11.8,1947.2,1,12,1533.076627,0.108108,79,18.5,17.0,17.5,23.5,18.0,17.0,21.0,13.5,15.5,12.0,-11.6,1829.2
0,7,1484.917968,0.416667,80,14.0,17.5,12.5,16.5,17.5,19.0,14.5,12.0,14.0,12.0,2.8,1821.9,1,14,1475.746749,0.125,260,16.5,12.0,20.0,18.5,11.0,20.5,18.0,11.5,12.5,16.5,-6.2,1952.6
0,13,1499.061026,0.565217,274,27.5,16.5,20.5,18.5,24.0,15.0,15.0,19.5,12.5,14.5,-27.0,2082.1,1,16,1424.270503,0.130435,274,15.0,18.0,8.5,14.0,16.0,27.5,18.5,27.0,16.5,13.5,22.6,1899.3


### 10. Predict Outcomes & Probabilities

In [504]:
#load the model in
model = pickle.load(open('aflModel.sav', 'rb'))
#predict the fixture
predictions = model.predict(fixtureReady)
probs = model.predict_proba(fixtureReady)
#find prob of results

#send to csv file
predictionDF = (predictionReady
                 #place in results
                 .with_columns(pl.Series(predictions).alias('Predictions'),
                               pl.Series(probs).alias('Probs'))
                 #Winner probs
                 .with_columns(((pl.col('Probs').list.get(pl.col('Predictions')))*100).round(2).alias('Win Prob'))
                 .with_columns(pl.when(pl.col('Predictions') == 1).then(pl.col('home.team.name')).otherwise(pl.col('away.team.name')).alias('Predicted Winner'))
                 #Add current round
                 .with_columns(pl.lit(roundNum).alias('Round Number'))
                 #rename
                 .rename({'home.team.name' : 'Home Team',
                          'away.team.name' : 'Away Team'})
                 #keep only wants required
                 .select('Round Number', 'Home Team', 'Away Team', 'Predicted Winner', 'Win Prob')
                )

print(predictionDF)

shape: (9, 5)
┌──────────────┬──────────────────┬───────────────────┬──────────────────┬──────────┐
│ Round Number ┆ Home Team        ┆ Away Team         ┆ Predicted Winner ┆ Win Prob │
│ ---          ┆ ---              ┆ ---               ┆ ---              ┆ ---      │
│ i32          ┆ str              ┆ str               ┆ str              ┆ f64      │
╞══════════════╪══════════════════╪═══════════════════╪══════════════════╪══════════╡
│ 11           ┆ Western Bulldogs ┆ Sydney Swans      ┆ Western Bulldogs ┆ 55.55    │
│ 11           ┆ Fremantle        ┆ Collingwood       ┆ Fremantle        ┆ 58.42    │
│ 11           ┆ North Melbourne  ┆ Port Adelaide     ┆ Port Adelaide    ┆ 55.75    │
│ 11           ┆ Carlton          ┆ Gold Coast Suns   ┆ Carlton          ┆ 63.34    │
│ 11           ┆ Geelong Cats     ┆ GWS Giants        ┆ Geelong Cats     ┆ 53.8     │
│ 11           ┆ Richmond         ┆ Essendon          ┆ Essendon         ┆ 56.04    │
│ 11           ┆ Hawthorn         ┆ Bris

### Add margins

In [505]:
import numpy

In [506]:
def margin(prob):
    prob = prob/100
    margin = round((-1*numpy.log((1-prob)/prob))/0.03, 0)
    if margin == 0:
        margin = 1
    return margin

In [510]:
predictionDF = predictionDF.with_columns(pl.col('Win Prob').map_elements(lambda x: margin(x)).alias('Margin')).unique()
print(predictionDF)
#send to csv file

shape: (9, 6)
┌──────────────┬──────────────────┬───────────────────┬──────────────────┬──────────┬────────┐
│ Round Number ┆ Home Team        ┆ Away Team         ┆ Predicted Winner ┆ Win Prob ┆ Margin │
│ ---          ┆ ---              ┆ ---               ┆ ---              ┆ ---      ┆ ---    │
│ i32          ┆ str              ┆ str               ┆ str              ┆ f64      ┆ f64    │
╞══════════════╪══════════════════╪═══════════════════╪══════════════════╪══════════╪════════╡
│ 11           ┆ Geelong Cats     ┆ GWS Giants        ┆ Geelong Cats     ┆ 53.8     ┆ 5.0    │
│ 11           ┆ Richmond         ┆ Essendon          ┆ Essendon         ┆ 56.04    ┆ 8.0    │
│ 11           ┆ Melbourne        ┆ St Kilda          ┆ Melbourne        ┆ 55.81    ┆ 8.0    │
│ 11           ┆ Adelaide Crows   ┆ West Coast Eagles ┆ Adelaide Crows   ┆ 77.38    ┆ 41.0   │
│ 11           ┆ Western Bulldogs ┆ Sydney Swans      ┆ Western Bulldogs ┆ 55.55    ┆ 7.0    │
│ 11           ┆ Fremantle        ┆ 

In [167]:
#send to csv file
predictionDF.write_csv(f'2024Predictions\Erin_Round_{roundNum}.csv')

round1.write_excel(f'2024Predictions\Erin_Round_1.xlsx', autofit=True)